In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from collections import defaultdict, Counter
from typing import List, Dict, Union

from tqdm import tqdm
import polars as pl

In [4]:
WINDOW_N = 10
WEIGHTS = {-10:0.01, -9:0.01, -8:0.01, -7:0.01, -6:0.01, -5:0.05, -4:0.05, -3:0.1, -2:0.25, -1:0.5, 1:3, \
2:0.5, 3:0.25, 4:0.1, 5:0.05, 6:0.01, 7:0.01, 8:0.01, 9:0.01, 10:0.01, }
TOP_N = 200
LOCALES = ["DE", "UK", "JP"]
VER = "25"
DIR = "/gdrive/MyDrive/amazon_kdd_2023/"


In [5]:
def preprocess(df:pl.DataFrame) -> pl.DataFrame:
    df = df.explode(["prev_items"])
    df = df.with_columns(
        df.select(pl.col("session_id").cumcount().over("session_id").alias("sequence_num"))
    )
    return df

In [6]:
def generate_co_visit_matrix(df:pl.DataFrame, window_n:int, weights:Dict[int, float]) -> pl.DataFrame:
    # 共起ペアの作成
    df = df[["session_id", "prev_items", "sequence_num"]].join(df[["session_id", "prev_items", "sequence_num"]], on="session_id")

    # 共起した間隔を計算し、絞り込み
    df = df.with_columns(
        (pl.col("sequence_num_right").cast(pl.Int64) - pl.col("sequence_num").cast(pl.Int64)).alias("diff_sequence_num")
    )
    df = df.filter(pl.col("diff_sequence_num").abs() <= window_n)
    df = df.filter(pl.col("prev_items") != pl.col("prev_items_right"))

    # weightを計算し、共起ペアごとに和を計算
    df = df.with_columns(
        pl.col("diff_sequence_num").map_dict(weights).alias("co_visit_weight")
    )
    df = df.groupby(["prev_items", "prev_items_right"]).sum()
    df = df.rename({"prev_items":"item", "prev_items_right":"candidate_item"})[["item", "candidate_item", "co_visit_weight"]]

    return df

In [7]:
def filter_by_locale_availability(co_visit_matrix:pl.DataFrame, product:pl.DataFrame):
    product = product.unique(subset=["id"])
    product = product[["id", "available_locales"]]
    co_visit_matrix = co_visit_matrix.join(product, left_on="item", right_on="id", how="left").rename({"available_locales":"item_locales"})
    co_visit_matrix = co_visit_matrix.join(product, left_on="candidate_item", right_on="id", how="left").rename({"available_locales":"candidate_item_locales"})
    dfs = []
    for locale in LOCALES:
        df = co_visit_matrix.filter(pl.lit(locale).is_in(pl.col("item_locales")) & pl.lit(locale).is_in(pl.col("candidate_item_locales"))) 
        df = df.with_columns(pl.lit(locale).alias("locale"))
        df = df[["item", "candidate_item", "co_visit_weight", "locale"]]
        df = df.sort(["item", "co_visit_weight"], descending=[False, True])
        df = df.groupby("item", maintain_order=True).head(TOP_N)
        df = df.with_columns(
            pl.col("co_visit_weight").rank(descending=True, method="min").over("item").alias("co_visit_rank")
        )
        dfs.append(df)
    co_visit_matrix = pl.concat(dfs)
    return co_visit_matrix

# For local train/eval

In [8]:
train1 = pl.read_parquet(DIR + "data/preprocessed/task1/train_task1.parquet")
train2 = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
train1 = train1.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
train = pl.concat([train1, train2])

test1_1 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase1.parquet")
test1_2 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet")
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet")
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet")
test1_1 = test1_1.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test1_2 = test1_2.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test1_1, test1_2, test2_1, test2_2, test3_1, test3_2])

In [9]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])

In [10]:
product = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/common/product_03.parquet")

In [11]:
co_visit_matrix = generate_co_visit_matrix(session_df, window_n=WINDOW_N, weights=WEIGHTS)

In [12]:
co_visit_matrix = filter_by_locale_availability(co_visit_matrix, product)

In [13]:
file_name = f"co_visit_matrix_{VER}_for_train_or_eval.parquet"
co_visit_matrix.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/" + file_name)

In [14]:
co_visit_matrix.head()

item,candidate_item,co_visit_weight,locale,co_visit_rank
str,str,f64,str,u32
"""0007440847""","""0008376107""",3.25,"""DE""",1
"""0007440847""","""3608938184""",0.5,"""DE""",2
"""0007440847""","""0261103563""",0.5,"""DE""",2
"""0007440847""","""3608937145""",0.5,"""DE""",2
"""0007440847""","""3608939849""",0.4,"""DE""",5


## MRR@100

In [15]:
train = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/task1/train_task1.parquet")

In [16]:
# last_itemの抽出
last_item_list = []
prev_items_list = train["prev_items"].to_list()
for prev_items in prev_items_list:
    last_item_list.append(prev_items[-1])
train = train.with_columns(pl.Series(name="last_item", values=last_item_list))

In [17]:
train = train[["session_id", "locale", "last_item", "next_item"]]

In [18]:
co_visit_matrix = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/" + file_name)

In [19]:
# candidateの結合とlabelの付与
dfs = []
label_lists = []
for locale in LOCALES:
    df = train.filter(pl.col("locale")==locale)
    matrix = co_visit_matrix.filter(pl.col("locale")==locale)
    df = df.join(matrix, left_on=["locale", "last_item"], right_on=["locale", "item"], how="left")
    df = df.sort(["session_id", "co_visit_weight"], descending=[False, True])
    df = df.with_columns((pl.col("candidate_item") == pl.col("next_item")).cast(pl.Int8).alias("label"))
    label_lists.extend(df.groupby("session_id", maintain_order=True).all()["label"].to_list())

In [20]:
# MRRの計算
rr = 0
for labels in label_lists:
    labels = labels[:100]
    for i, label in enumerate(labels):
        if label == 1:
            rr += 1 / (i+1)
            break
mrr = rr / len(label_lists)
print("MRR:", round(mrr, 5))

MRR: 0.23473


# For test inference

In [8]:
train1 = pl.read_parquet(DIR + "data/preprocessed/task1/train_task1.parquet")
train2 = pl.read_parquet(DIR + "data/preprocessed/task2/train_task2.parquet")
train1 = train1.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
train = pl.concat([train1, train2])

test1_1 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase1.parquet")
test1_2 = pl.read_parquet(DIR + "data/preprocessed/task1/test_task1_phase2.parquet")
test2_1 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase1.parquet")
test2_2 = pl.read_parquet(DIR + "data/preprocessed/task2/test_task2_phase2.parquet")
test3_1 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase1.parquet")
test3_2 = pl.read_parquet(DIR + "data/preprocessed/task3/test_task3_phase2.parquet")
test1_1 = test1_1.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test1_2 = test1_2.with_columns(
    (pl.col("session_id") + "_from_task1").alias("session_id")
)
test3_1 = test3_1.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test3_2 = test3_2.with_columns(
    (pl.col("session_id") + "_from_task3").alias("session_id")
)
test = pl.concat([test1_1, test1_2, test2_1, test2_2, test3_1, test3_2])

In [9]:
# trainのnext_itemをprev_itemsにappendする
prev_items_list = train["prev_items"].to_list()
next_item_list = train["next_item"].to_list()
prev_items_list_updated = []
for prev_items, next_item in zip(prev_items_list, next_item_list):
    prev_items.append(next_item)
    prev_items_list_updated.append(prev_items)

train = train.with_columns(
    pl.Series(name="prev_items", values=prev_items_list_updated)
)

In [10]:
train = preprocess(train)
test = preprocess(test)
session_df = pl.concat([
    train["prev_items", "locale", "session_id", "sequence_num"],
    test["prev_items", "locale", "session_id", "sequence_num"],
])

In [11]:
product = pl.read_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/preprocessed/common/product_03.parquet")

In [12]:
co_visit_matrix = generate_co_visit_matrix(session_df, window_n=WINDOW_N, weights=WEIGHTS)

In [13]:
co_visit_matrix = filter_by_locale_availability(co_visit_matrix, product)

In [ ]:
file_name = f"co_visit_matrix_{VER}_for_inference.parquet"
co_visit_matrix.write_parquet("/gdrive/MyDrive/amazon_kdd_2023/data/interim/candidates/task1/" + file_name)

In [ ]:
co_visit_matrix.head()